In [1]:
pip install qiskit


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install qiskit_machine_learning

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from qiskit_machine_learning.algorithms import QSVC

In [4]:
from qiskit_machine_learning.kernels import FidelityQuantumKernel

In [12]:
import pandas as pd
from qiskit_algorithms.utils import algorithm_globals
algorithm_globals.random_seed = 123
df = pd.read_csv('cardio_train.csv',sep=';',nrows=500)
df

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,711,21791,1,150,88.0,120,80,1,1,0,0,1,0
496,712,17381,1,151,117.0,160,90,2,1,0,0,0,1
497,713,20449,1,151,53.0,120,80,1,1,0,0,1,0
498,714,21197,1,162,60.0,120,80,1,1,0,0,0,0


In [13]:
from sklearn.model_selection import train_test_split
from qiskit_algorithms.utils import algorithm_globals

algorithm_globals.random_seed = 123
train, test = train_test_split(
    df, train_size=0.8, random_state=algorithm_globals.random_seed)

In [14]:
X_train= train[['ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco']]
Y_train=train['cardio']
X_test=test[['ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco']]
Y_test=test['cardio']

In [15]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(400, 6)
(100, 6)
(400,)
(100,)


In [16]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
num_features = X_train.shape[1]
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=2, entanglement="linear")

sampler = Sampler()

fidelity = ComputeUncompute(sampler=sampler)

kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)

C:\Users\admin\AppData\Local\Temp\ipykernel_13672\990892341.py:8: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()


In [17]:
from qiskit_machine_learning.algorithms import QSVC

qsvc = QSVC(quantum_kernel=kernel)
qsvc.fit(X_train, Y_train)
qsvc_score = qsvc.score(X_test, Y_test)
qsvc1_score= qsvc.score(X_train,Y_train)
print(f"QSVC classification train score: {qsvc1_score}")
print(f"QSVC classification test score : {qsvc_score}")

QSVC classification train score: 0.8025
QSVC classification test score : 0.69


In [18]:
import pickle
with open('qsvc_model.pkl', 'wb') as file:
    pickle.dump(qsvc, file)